# Data Load

In [1]:
import sentencepiece as spm
import torch

en_vocab_file = "../data preprocess/en.model"
de_vocab_file = "../data preprocess/de.model"

en_vocab = spm.SentencePieceProcessor()
de_vocab = spm.SentencePieceProcessor()

en_vocab.load(en_vocab_file)
de_vocab.load(de_vocab_file)

True

In [2]:
# Example

lines = [
  "summer is leavning",
  "winter is coming"
]

inputs = []
for line in lines:
  pieces = en_vocab.encode_as_pieces(line)
  ids = en_vocab.encode_as_ids(line)
  inputs.append(torch.tensor(ids))
  print(pieces)

# 입력 최대 길이에 맞춰 0으로 패딩
inputs = torch.nn.utils.rnn.pad_sequence(inputs, batch_first=True, padding_value=0)
print(inputs.size())
print(inputs)

['▁summer', '▁is', '▁le', 'av', 'ning']
['▁winter', '▁is', '▁coming']
torch.Size([2, 5])
tensor([[3299,   54,  293,  638,  548],
        [5026,   54, 1079,    0,    0]])


In [8]:
import pandas as pd

train_df = pd.read_csv('train_df.csv')
train_df['en'][0]


'Thank you so much, Chris.'

In [10]:
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.distributed import DistributedSampler


class en2deDataset(Dataset):
    # 생성자 및 데이터 전처리
    def __init__(self, dataframe, src_vocab, tgt_vocab):
        self.src_vocab = src_vocab
        self.tgt_vocab = tgt_vocab

    # 데이터셋의 총 길이
    def __len__(self):
        return len(self.dataframe)
    
    # idx에 해당하는 입출력 데이터 리턴
    def __getitem__(self, idx):
        src_sentence = self.dataframe['en'][idx]
        tgt_sentence = self.dataframe['de'][idx]
        
        scr_enc = self.src_vocab.encode_as_ids(src_sentence)
        tgt_enc = self.tgt_vocab.encode_as_ids(src_sentence)

        return torch.tensor(scr_enc), torch.tensor(tgt_enc)
# src_vocab 인코딩, tgt_vocab 인코딩을 텐서로 변환하여 튜플 형태로 return



In [ ]:
Class en2deDataloader():


    def collate_fn(batch, pad):
        enc_inputs = torch.nn.utils.rnn.pad_sequence(enc_inputs, batch_first=True)
        dec_inputs = torch.nn.utils.rnn.pad_sequence(dec_inputs, batch_first=True)